In [1]:
import pandas as pd
import numpy as np
import yaml
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, plot_estimation

##### 1. Load all data that is needed

In [2]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [3]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

##### 2. Define all possible equations for mu and qs

The growth rate and the substrate uptake rate are dependend of each other - meaning that if we define a equation for **mu**, then **qs** is defined by mu with $qs=mu/Yxs$. Contrarily, when **qs** is defined by an ODE, then **mu** is calculated with $mu = qs * Yxs$.

In [4]:
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks) # -- MONOD
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) # -- LOGISTIC
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC + INHIBITION
mu4 = lambda qs, Yxs: qs * Yxs # for changing qs

In [5]:
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose) # -- MONOD
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met)) # -- MONOD + NON COMPETITIVE INHIBITION
qs2 = lambda qs_max, c_glucose, Ks_qs, c_biomass, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(c_biomass * lag))) # -- MONOD + METABOLIZED GLU
qs3 = lambda mu, Yxs: mu / Yxs # for changing mu

In [6]:
mu_all=[mu0, mu1, mu2, mu3, mu4]
qs_all=[qs0, qs1, qs2, qs3]

##### 3. Define functions that are needed

In [7]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

##### 4. Define set of parameters in config file

In [8]:
## not needed anymore
# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']
est_Ks = param['est_ks']

In [9]:
# Set the number of samples and parameters
num_samples = 50
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.1, 10.0],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.2, 0.6],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.15],   # Range for parameter 8 - m_s
    [0.001, 0.02],  # Range for parameter 9 - lag
]

In [10]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(50, 9)

##### 5. Run it

In [11]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        # Define the combination of equations
        mu_eq = mu_all[i]; num_mu = i

        # mu 0 to mu3 are combined with qs3 while for mu4 all qs except for qs3 are combined
        if i != 4:
            j = 3
            print(i,j)
            qs_eq = qs_all[j]; num_qs = j   

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(5):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
        
        elif i == 4:
            for j in range(3):
                print(i,j)
                qs_eq = qs_all[j]; num_qs = j   

                # Make the predictions and calculate the error
                rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
                # save the parameters in a dataframe
                append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
                rmse_one_set.append(append_list)

                # Make a plot and save it
                title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
                plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

                if rmse <= float(5):
                    plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/new_eq/data.csv')

df_all_sets.head(10)

0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0
4 1
4 2
0 3
1 3
2 3
3 3
4 0


,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
268,38,2,3,0.377683,22.281250,5.953627,7.097474,7.822499,0.205584,1.185225,0.066972,0.012776,3.864
324,46,2,3,0.343094,22.403138,8.775724,2.609584,6.382724,0.220889,1.417867,0.122510,0.016065,4.873
226,32,2,3,0.493708,21.663367,0.984248,6.430593,7.977904,0.312137,0.725232,0.147715,0.008585,5.768
254,36,2,3,0.574696,20.715424,2.204429,3.691370,6.225326,0.266802,0.701289,0.098461,0.002496,5.844
323,46,1,3,0.343094,22.403138,8.775724,2.609584,6.382724,0.220889,1.417867,0.122510,0.016065,6.340
65,9,2,3,0.429549,24.956187,1.819280,2.223970,7.581630,0.292185,1.003394,0.104867,0.006515,6.447
344,49,1,3,0.304393,20.969525,8.016063,8.274238,7.447213,0.263090,0.901380,0.142395,0.005735,6.510
240,34,2,3,0.580124,22.553268,5.368785,7.813412,7.846110,0.301468,1.234326,0.064305,0.018557,7.057
198,28,2,3,0.556259,23.985043,9.037730,8.598081,6.246747,0.226751,1.082145,0.076449,0.013823,7.077
204,29,1,3,0.202083,23.830951,3.207570,2.833499,6.692741,0.232825,1.132796,0.054173,0.004892,7.135


In [12]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')